## **Import necessary stuff**

In [2]:
# Imports
import pandas as pd
import numpy as np
from statistics import mean

# **Bayes Theorem**

Create a Colab script which calculates P(A|B) using Bayes theorem for the example provided in the lecture; that is, if you test positive how probable is it that you have cancer?

*   Create a Python script to implement the bayes_theorem  function

*   Create a runner to call your function and output the result,  P(A|B) for the following cases:
  * 1:  Cancer occurs in 1% of the people your age and the test is 99% reliable
  * 2:  Cancer occurs in 3% of the people your age and the test is 90% reliable
  * Repeat the tests 3 times each (using your previous P(A|B) result in the subsequent run) and record P(A|B) for each repeated test.  How did your results improve with each additional test run?




```
def bayes_theorem(set the parameters as you see fit):
  #YOUR CODE HERE
  return p_a_given_b
```



In [3]:
def bayes_theorem(p_a, p_b_given_a):
  #YOUR CODE HERE

  true_positive = p_a * p_b_given_a
  false_positive = (1-p_a) * (1-p_b_given_a)

  p_a_given_b = true_positive/(true_positive + false_positive)

  return p_a_given_b

In [4]:
bayes = 0.01
print("Cancer occurs in 1% of the people your age and the test is 99% reliable")
for i in range(4):
  bayes = bayes_theorem(bayes, 0.99)
  print("Round", i, ": ", bayes)

print("\n\n")

bayes = 0.03
print("Cancer occurs in 3% of the people your age and the test is 90% reliable")
for i in range(4):
  bayes = bayes_theorem(bayes, 0.9)
  print("Round", i, ": ", bayes)

Cancer occurs in 1% of the people your age and the test is 99% reliable
Round 0 :  0.4999999999999997
Round 1 :  0.99
Round 2 :  0.9998979800040808
Round 3 :  0.99999896939091



Cancer occurs in 3% of the people your age and the test is 90% reliable
Round 0 :  0.21774193548387102
Round 1 :  0.7147058823529412
Round 2 :  0.9575306479859894
Round 3 :  0.9950960566228514


# **Naive Bayes - Monomial Classifier**
Create a Colab script which uses a Naive Bayes classifier to detect email spam messages.  
* Use the message ‘subject’ line (you can use the body but it’ll be much more complex)
* Create a training set of Wanted and Unwanted messages
* Create your ‘model’ 
   * Parse the words in the emails and create conditional probabilities for the Wanted and Unwanted messages
* Test your model with other messages not used in the training set
* Record and reflect on your results, i.e. evaluate the performance, and compare against typical performance of the algorithm (do some searches to find the average performance of a Naive Bayes based spam filter).
* Several training datasets have been provided as materials for this assignment.  You may find another, or scrape your own emails if you prefer - just describe what dataset you used or created in your reflections.



In [5]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ML_S23-Mauro/MachineLearning/BayesTheorem'

Mounted at /content/drive
/content/drive/MyDrive/ML_S23-Mauro/MachineLearning/BayesTheorem


In [6]:
df = pd.read_csv("/content/drive/MyDrive/ML_S23-Mauro/MachineLearning/BayesTheorem/spam_training_dataset.csv")


df['Subject'] = df.text.str.split('\r').str.get(0)
label = list(df.label)

HAM = 'ham'
SPAM = 'spam'

cutoff = int(len(df['Subject'])*.8)
trainData = df['Subject'][:cutoff]
testData = df['Subject'][cutoff:]
wantedDict = {}
unwantedDict = {}

for i in range(len(df['Subject'])):
  subject = df['Subject'][i].strip("Subject: ")
  for word in subject.split(" "):
    if word.isalpha():
      if((word not in wantedDict) or (word not in unwantedDict)):
          wantedDict[word] = 1
          unwantedDict[word] = 1

      if(df.iloc[i, 1] == HAM):
        if(word not in wantedDict):
          wantedDict[word] = 1
        else:
          wantedDict[word] = (wantedDict[word] + 1)
      elif(df.iloc[i, 1] == SPAM):
        if(word not in unwantedDict):
          unwantedDict[word] = 1
        else:
          unwantedDict[word] = (unwantedDict[word] + 1)


global wantedTotalPossiblities, unwantedTotalPossiblities, wantedInitialProb, unwantedInitialProb

wantedTotalWords = sum(wantedDict.values())
unwantedTotalWords = sum(unwantedDict.values())
wantedInitialProb = label.count('ham')/len(label)
unwantedInitialProb = label.count('spam')/len(label)
correctGuess = []


for i in range(len(testData)):
  subject = testData[i+4136].strip("Subject: ")
  filter = df.iloc[i, 1]

  wantedScore = wantedInitialProb
  unwantedScore = unwantedInitialProb
  for word in subject.split():
    if word.isalpha():
      wantedScore *= (wantedDict.get(word, 0) + 1) / (wantedTotalWords + len(wantedDict))
      unwantedScore *= (wantedDict.get(word, 0) + 1) / (wantedTotalWords + len(wantedDict))
  ham_or_spam = HAM if(wantedScore > unwantedScore) else SPAM
  correctGuess.append(1 if (ham_or_spam == filter) else 0)

print("The filter has an accuracy of:", mean(correctGuess))

The filter has an accuracy of: 0.7043478260869566


# **Part 3:  Naive Bayes - Gaussian Classifier**
*   Create a Colab script which uses a Naive Bayes classifier to predict one of the 3 species of flowers based on input attributes from the Iris dateset
*   Create an 80/20 split of the dataset
*   Generate the Gaussian distributions for each of your attributes in each of the classes (species) for your training data
*   Use the test data to predict the species
*   Calculate and record the accuracy of your classifier
*   The Iris dataset has been provided as materials for this assignment.  You may also download the dataset from: [https://archive.ics.uci.edu/ml/machine-learning-databases/iris/](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/)







In [73]:
import pandas as pd
import numpy as np
from statistics import stdev
import math

def calculate_likelihood(value, mean, std):
    exponent = math.exp(-(math.pow(value - mean, 2) / (2 * math.pow(std, 2))))
    return (1 / (math.sqrt(2 * math.pi) * std)) * exponent


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'flower-type']
flowers_DataFrame = pd.read_csv(url, names=names)

setosa = flowers_DataFrame[flowers_DataFrame['flower-type'] == 'Iris-setosa']
versicolor = flowers_DataFrame[flowers_DataFrame['flower-type'] == 'Iris-versicolor']
virginica = flowers_DataFrame[flowers_DataFrame['flower-type'] == 'Iris-virginica']

setosaCutoff = int((len(setosa)*0.8))
versicolorCutoff = int((len(versicolor)*0.8))
virginicaCutoff = int((len(virginica)*0.8))

irisTrainingData = pd.concat([setosa[:setosaCutoff], versicolor[:versicolorCutoff], virginica[:virginicaCutoff]])
irisTestingData = pd.concat([setosa[setosaCutoff:], versicolor[versicolorCutoff:], virginica[virginicaCutoff:]])

renameRows = []
for i in range(len(irisTrainingData)):
  renameRows.append(i)
irisTrainingData.index = renameRows
renameRows = []
for i in range(len(irisTestingData)):
  renameRows.append(i)
irisTestingData.index = renameRows


grouped = irisTrainingData.groupby('flower-type')
gaussians = {}
for name, group in grouped:
    gaussians[name] = {}
    for col in irisTrainingData.columns[:-1]:
        gaussians[name][col] = {
            'mean': np.mean(group[col]),
            'std': np.std(group[col])
        }

predictions = []
for index, row in irisTestingData.iterrows():
    likelihoods = {}
    for name in gaussians.keys():
        likelihood = 1
        for col in irisTrainingData.columns[:-1]:
            likelihood *= calculate_likelihood(row[col], gaussians[name][col]['mean'], gaussians[name][col]['std'])
        likelihoods[name] = likelihood
    predictions.append(max(likelihoods, key=likelihoods.get))

irisTestingData['predictions'] = predictions

accuracy = sum(irisTestingData['flower-type'] == irisTestingData['predictions']) / len(irisTestingData)
print("Accuracy: {:.2f}".format(accuracy))

Accuracy: 1.00
